<a href="https://colab.research.google.com/github/trudramukerji14/DonnieDarkoVision/blob/main/Donnie_Darko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an use of

In [2]:
!pip install langchain
!pip install openai
#openai dependencies
!pip install tiktoken
!pip install astrapy
# Install python-dotenv
!pip install python-dotenv
#needed
!pip install pypdf



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import requests
#from bs4 import BeautifulSoup
import os
import re

We first scrape a pdf of the script of **Donnie Darko** available at the url below using response.

In [4]:
folder_path = './pdfs'  # Replace with your local folder path
if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    print("Directory '%s' created" % folder_path)
else:
    print("Directory '%s' already exists" % folder_path)



link = 'https://www.dailyscript.com/scripts/donniedarko.pdf'
response = requests.get(link)
pdf_url = link
pdf_name = pdf_url.split('/')[-1]
pdf_response = requests.get(link)
with open(os.path.join(folder_path, pdf_name), 'wb') as f:
  f.write(pdf_response.content)

Directory './pdfs' created


We now load an .env file which contains all our API keys as variable information. To do this, we


1.   Mount Google Drive
2.   From the Drive, we copy the file path of the .env file we are using.
3.   We then load our variable information using dotenv






In [5]:
from google.colab import drive

drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [6]:
import dotenv
import os

driveenv_file_path = '' #insert file path of .env from your mounted drive here
dotenv.load_dotenv(driveenv_file_path )
ASTRA_DB_APPLICATION_TOKEN = os.environ.get("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_API_ENDPOINT = os.environ.get("ASTRA_DB_API_ENDPOINT")
OPEN_AI_API_KEY = os.environ.get("OPENAI_API_KEY")
ASTRA_DB_COLLECTION = "test"


We now use OpenAIEmbeddings to create vector representations of the text contained within the pdf we just scraped. That in turn is stored in a vector database. The vector database we use is provided by [DataStax.](https://www.datastax.com/products/datastax-astra)

In [7]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores.astradb import AstraDB
from langchain.embeddings import OpenAIEmbeddings




In [8]:
embedding = OpenAIEmbeddings()
vstore = AstraDB(
    embedding=embedding,
    collection_name="test",
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
    api_endpoint=os.environ["ASTRA_DB_API_ENDPOINT"],
)

folder_path = './pdfs'
loader = PyPDFDirectoryLoader("./pdfs")
docs = loader.load_and_split()

inserted_ids = vstore.add_documents(docs)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.astradb.AstraDB` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. An updated version of the class exists in the langchain-astradb package and should be used instead. To use it run `pip install -U langchain-astradb` and import as `from langchain_astradb import AstraDBVectorStore`.
  warn_deprecated(


We now implement the RAG chatbot. We set a retriever that interacts with the LLM provided by OpenAI's API to augment the query.

In [11]:
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


retriever = vstore.as_retriever(search_kwargs={'k': 3}) #returns k most similar vectors.

prompt_template = """
You are a screenwriter with deep knowledge of the characters described in the context. Write scenes for the characters based on the question.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI(openai_api_key=OPEN_AI_API_KEY)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

response = chain.invoke("What would happen if Donnie had lunch at OpenAI?")
print(response)

INT. OPENAI CAFETERIA - DAY

Donnie walks into the bustling cafeteria at OpenAI, looking around curiously at the various tech-savvy individuals chatting and eating together. He grabs a tray and starts to move along the food stations, contemplating his choices.

DONNIE
(to himself)
I wonder if time travel discussions are on the menu today.

As he reaches the end of the line, he spots a familiar face sitting alone at a table. It's Dr. Monnitoff, engrossed in a book on quantum mechanics. Donnie hesitates for a moment before deciding to join him.

DONNIE
Hey, Dr. Monnitoff. Mind if I join you?

Dr. Monnitoff looks up, surprised to see Donnie but nods with a smile. They start discussing the latest advancements in AI technology and its potential implications for the future.

DR. MONNITOFF
It's fascinating how AI is reshaping our understanding of intelligence and consciousness.

DONNIE
It's like creating a whole new realm of possibilities, like a parallel universe of knowledge.

Their convers